# **Importing all required libraries**

In [2]:
import pandas as pd
import numpy as np
from datetime import date

# **1. Loading Dataset**
* I downloaded the CSV file named mock_data.csv from the given URL.
* I will read the file into a Pandas DataFrame and load only the first 75% of the rows.

In [3]:
#Loading the dataset
df = pd.read_csv('/content/MOCK_DATA (3).csv')

df = df.iloc[:int(0.75 * len(df))] #This loads only the first 75% of the rows
print(f'len(df) : {len(df)}')
df.head()

len(df) : 750


,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29.000000000
1,1029,China,47,2016-01-08 03:13:12.000000000
2,1025,Nigeria,10,2023-02-20 19:06:14.000000000
3,1006,United Kingdom,48,2022-05-26 12:46:35.000000000
4,1024,Philippines,16,2016-12-13 03:44:04.000000000


# **2. Data Cleaning**
1. I will remove duplicate rows but only if they appear in consecutive order.

2. I will convert the "Sign-up Date" column to a proper datetime format and replace all entries from the year 2020 with NaN.
3. I will drop all rows where "Total Purchases" is less than 5, but only if the customer is from Canada.

In [5]:
#Removing duplicate rows only if they appear in consecutive order
'''
df.shift() will shift all rows down by one
df.ne(df.shift()) will compare each row with the row above it
df.any(axis=1) will return True for any row that has at least one element that is different from the row above it
'''

df=df[df.ne(df.shift()).any(axis=1)].reset_index(drop=True)
print(f'len(df) : {len(df)}')

len(df) : 750


In [6]:
#Convert the "Sign-up Date" column to a proper datetime format, but replace all entries from the year 2020 with NaN.
df['Sign-up Date'] = pd.to_datetime(df['Sign-up Date'], errors='coerce')    #convert invalid or unparseable date values into NaT (Not a Time) instead of raising an error.

df.loc[df['Sign-up Date'].dt.year == 2020, 'Sign-up Date'] = np.nan
df_nan=df[df['Sign-up Date'].isna()]
print(f'len(df_nan) : {len(df_nan)}')
df_nan.head()

len(df_nan) : 88


,Customer ID,Country,Total Purchases,Sign-up Date
8,1029,Argentina,14,NaT
9,1014,China,3,NaT
15,1026,Serbia,35,NaT
21,1018,Russia,5,NaT
48,1014,Brazil,45,NaT


In [7]:
#Drop all rows where the column "Total Purchases" is less than 5 but only if the customer is from Canada.
df = df.drop(df[(df['Total Purchases'] < 5) & (df['Country'] == 'Canada')].index) # .index will extract the index values (row labels) of the filtered rows.
print(f'len(df) : {len(df)}')
df.head()

len(df) : 748


,Customer ID,Country,Total Purchases,Sign-up Date
0,1015,Palestinian Territory,12,2022-04-03 21:42:29
1,1029,China,47,2016-01-08 03:13:12
2,1025,Nigeria,10,2023-02-20 19:06:14
3,1006,United Kingdom,48,2022-05-26 12:46:35
4,1024,Philippines,16,2016-12-13 03:44:04


# **3. Feature Engineering**
I will create a new column called "Loyalty Score" based on the following conditions:


*   If Total Purchases > 20, I will assign a score of 3.
*   If Total Purchases is between 10 and 20, I will assign a score of 2.
If Total Purchases < 10, I will assign a score of 1, except if the customer has

*   If Total Purchases < 10, I will assign a score of 1, except if the customer has been a member for more than 3 years (using 'Sign-up Date')—then I will assign a score of 2 instead.

In [8]:
current_year = pd.Timestamp.today().year    #this will get year from today's date

'''
current_year-df['Sign-up Date'].dt.year)>3 will check customer that has been a member for more than 3 years
Customers with  Total Purchases < 10 and has been a member for more than 3 years will be assigned score of 2
I will make 2 lists, one for conditions and one for values, then use select function of numpy to group them
'''

conditions =[(df['Total Purchases']>20),
             (((df['Total Purchases']>=10) & (df['Total Purchases']<=20))|((df['Total Purchases']<10) & (current_year-df['Sign-up Date'].dt.year)>3)),
             ((df['Total Purchases']<10) & (current_year-df['Sign-up Date'].dt.year)<3)]

values = ['3', '2', '1']

df['Loyalty Score'] = np.select(conditions, values, default='Unknown')
df.head()

,Customer ID,Country,Total Purchases,Sign-up Date,Loyalty Score
0,1015,Palestinian Territory,12,2022-04-03 21:42:29,2
1,1029,China,47,2016-01-08 03:13:12,3
2,1025,Nigeria,10,2023-02-20 19:06:14,2
3,1006,United Kingdom,48,2022-05-26 12:46:35,3
4,1024,Philippines,16,2016-12-13 03:44:04,2


# **4. Data Aggregation & Filtering**




*   I will group the data by "Country" and calculate the average and maximum "Total Purchases."
* I will determine the most common Loyalty Score for each country, excluding Mexico entirely from the analysis.




In [9]:

df2 = df[df['Country']!='Mexico']   #this will exclude Mexico from the data
"""
Aggregates customer data by 'Country' to compute:
- The average and maximum 'Total Purchases'.
- The most common 'Loyalty Score' using mode().

Returns:
    DataFrame with columns: 'Country', 'avg_purchases', 'max_purchases',
    and 'most_common_loyalty_score'.
"""

output = df2.groupby('Country').agg(
    avg_purchases=('Total Purchases', 'mean'),
    max_purchases=('Total Purchases', 'max'),
    most_common_loyalty_score=('Loyalty Score', lambda x: x.mode().iloc[0])
).reset_index()

output.head()

,Country,avg_purchases,max_purchases,most_common_loyalty_score
0,Afghanistan,8.000000,20,1
1,Albania,20.500000,38,1
2,Andorra,11.000000,11,2
3,Angola,15.000000,29,1
4,Argentina,24.692308,45,3


# **5. Data Export & Report Generation**



*   I will save the final processed DataFrame as a CSV file
 then, I will name the file using the format "processed_data_YYYY_MM_DD.csv", where the date is written in reverse order (DD-MM-YYYY).
* I will ensure that only the columns ["Customer ID", "Loyalty Score", "Country", "Total Purchases"] are included in the final file
then print the first 10 rows of the processed DataFrame after shuffling them (without sorting).



In [10]:
# Select the desired columns
selected_columns = ['Customer ID', 'Loyalty Score', 'Country', 'Total Purchases']
df_selected = df[selected_columns]

# Shuffle the DataFrame
df_shuffled = df_selected.sample(frac=1, random_state=40) # random_state for reproducibility

# Print the first 10 rows
processed_data  = df_shuffled.head(10)
print(processed_data)


     Customer ID Loyalty Score         Country  Total Purchases
429         1033             1           China                3
364         1003             2          Mexico               14
209         1024             3        Bulgaria               24
370         1015             2          Russia               18
573         1011             3   United States               39
132         1031             3         Ireland               31
340         1009             3           China               22
412         1035             1  Czech Republic                6
386         1018             1         Ecuador                5
213         1021             3           Niger               30


In [13]:
# Get today's date
today = date.today()

# Format the date as DD-MM-YYYY for export
date_string = today.strftime("%d-%m-%Y")

# Create the filename
filename = f"processed_data_{date_string}.csv"

# Save the DataFrame to the CSV file
processed_data.to_csv(filename, index=False)

# **Summary of Findings**
The MOCK_DATA dataset contains 1000 rows in total, loading 75% left us with 750 rows. There are no duplicate rows that appear in consecutive order as the size of the data is the same even after dropping duplicate with consecutive order. There were 2 customers from canada with "Total Purchases less than 5.

A new column 'Loyalty Score' was created which assign '1', '2' and '3' to the customers based on their Total purchases and in some cases how long they have been a member.

From the first 5 rows of the data after aggregation, Afghanistan, Albania and Angola have very low  most common Loyalty score of	1 suggesting a majority of customers from those countries have fewer purchases or shorter membership durations.

From the shuffled final data, United State has the highest Total Purchases with very high Loyalty Score.
